In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lmfit

%run "C:\Users\conde\Desktop\My Archives\Codes\Python\Data Analysis\my_optical_analysis_functions.ipynb"

In [ ]:
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\LTOI e LN on Sapphire\\LTOI\\LTOI - Simulation"

directory_file_list = os.listdir(directory)
selected_files = []
counter = 0

for i in directory_file_list:
    if i.endswith(".csv"):
        print(f"{counter}: {i}")
        selected_files.append(i)
        counter += 1

In [ ]:
idx_file = 0
df_delimiter = ','
df_header = 4

file_name = selected_files[idx_file]
df_raw = pd.read_csv(directory+"\\"+selected_files[idx_file], delimiter=df_delimiter, header=df_header)
df_raw = df_raw.map(lambda x: complex(str(x).replace('i', 'j')))
columns_raw=df_raw.columns

print(file_name)
df_raw.head()

In [ ]:
idx_df_columns_to_round = [0,1,2]
idx_df_columns_to_delete = []
real_digits = 10
imag_digits = 10
#----------

df = pd.DataFrame(columns=columns_raw)

for i in columns_raw:
    df[i] = df_raw[i]

for i in idx_df_columns_to_round:
    df[columns_raw[i]] = np.round(df_raw[columns_raw[i]].values.real, real_digits) + np.round(df_raw[columns_raw[i]].values.imag, imag_digits)

for i in idx_df_columns_to_delete:
    del df[columns_raw[i]]

columns=df.columns

print(file_name)
df.head()

In [ ]:
print(file_name)
counter = 0
for i in columns:
    print(f"{counter}: {i}")
    counter += 1

In [59]:
df_filtered = pd.DataFrame(columns=df.columns)
for i in columns:
    df_filtered[i] = df[i]
    
df_filtered = df_filtered[df_filtered[columns[0]].values.real*180/np.pi>64.8]
df_filtered = df_filtered[df_filtered[columns[1]].values.real>59e-9]
df_filtered = df_filtered[df_filtered[columns[9]].values.real>1.55]
# df_filtered = df_filtered[df_filtered[columns[2]].values.real>29e-9]

#df_filtered = df_filtered[df_filtered[columns[12]].values.real>0.99]
df_filtered_TE = df_filtered[df_filtered[columns[4]].values.real>0.1]
df_filtered_TM = df_filtered[df_filtered[columns[6]].values.real>0.1]

In [ ]:
idx_df_column_X = 13
X_TE = df_filtered_TE[columns[idx_df_column_X]].values.real
X_TM = df_filtered_TM[columns[idx_df_column_X]].values.real

idx_df_column_Y = 7
Y_TE = df_filtered_TE[columns[idx_df_column_Y]].values.real/(2*np.pi)
Y_TM = df_filtered_TM[columns[idx_df_column_Y]].values.real/(2*np.pi)

fig, ax = plt.subplots(1,2,figsize=(12,4))

ax[0].scatter(X_TE,Y_TE)
ax[0].set_xlabel(columns[idx_df_column_X])
ax[0].set_ylabel(columns[idx_df_column_Y])
ax[0].grid(True)

ax[1].scatter(X_TM,Y_TM)
ax[1].set_xlabel(columns[idx_df_column_X])
ax[1].set_ylabel(columns[idx_df_column_Y])
ax[1].grid(True)

print(file_name)

In [61]:
lambda_sweep = 'down'
freq0 = 193e12
m0 = 700

if lambda_sweep=='up':
    X_TE_mod = np.flip(X_TE) - m0
    X_TM_mod = np.flip(X_TM) - m0
    Y_TE_mod = np.flip(Y_TE) - freq0
    Y_TM_mod = np.flip(Y_TM) - freq0
elif lambda_sweep=='down':
    X_TE_mod = X_TE - m0
    X_TM_mod = X_TM - m0
    Y_TE_mod = Y_TE - freq0
    Y_TM_mod = Y_TM - freq0

In [ ]:
X_TE_int = np.arange(math.ceil(X_TE_mod[0]), math.floor(X_TE_mod[-1]),1)
Y_TE_int = interp1d(X_TE_mod, Y_TE_mod)(X_TE_int)

X_TM_int = np.arange(math.ceil(X_TM_mod[0]), math.floor(X_TM_mod[-1]),1)
Y_TM_int = interp1d(X_TM_mod, Y_TM_mod)(X_TM_int)

fig, ax = plt.subplots(1,2,figsize=(12,4))
fig.tight_layout()

ax[0].scatter(X_TE_int,Y_TE_int)
ax[0].grid(True)

ax[1].scatter(X_TM_int,Y_TM_int)
ax[1].grid(True)

print(file_name)

In [ ]:
X_TE_disp = X_TE_int
Y_TE_disp = Y_TE_int

model = lmfit.Model(freq_disp)
params = model.make_params(D0={'value': 5.0e12, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D1={'value': 170.0e9, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D2={'value': -50.0e6, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D3={'value': 1.0e7, 'vary': True, 'min':-np.inf, 'max':np.inf})
result = model.fit(Y_TE_disp, params, u=X_TE_disp, method='leastsq')

ans_TE = result.params.valuesdict()

fig, ax = plt.subplots(figsize=(13,4))
fig.tight_layout()
ax.scatter(X_TE_disp, (Y_TE_disp - ans_TE['D0'] - X_TE_disp*ans_TE['D1']), color='black', linewidth=3)
ax.plot(X_TE_disp, freq_disp(X_TE_disp, 0, 0, ans_TE['D2'], ans_TE['D3']), color='red', linewidth=2, linestyle='dashed')
ax.grid(True)
print(file_name)

caique_plot(X_TE_disp, Y_TE_disp, plottype='scatter', annotate=False, result=result)

In [ ]:
X_TM_disp = X_TM_int
Y_TM_disp = Y_TM_int

model = lmfit.Model(freq_disp)
params = model.make_params(D0={'value': 5.0e12, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D1={'value': 170.0e9, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D2={'value': -50.0e6, 'vary': True, 'min':-np.inf, 'max':np.inf},
                           D3={'value': 1.0e7, 'vary': True, 'min':-np.inf, 'max':np.inf})
result = model.fit(Y_TM_disp, params, u=X_TM_disp, method='leastsq')

ans_TM = result.params.valuesdict()

fig, ax = plt.subplots(figsize=(13,4))
fig.tight_layout()
ax.scatter(X_TM_disp, (Y_TM_disp - ans_TM['D0'] - X_TM_disp*ans_TM['D1']), color='black', linewidth=3)
ax.plot(X_TM_disp, freq_disp(X_TM_disp, 0, 0, ans_TM['D2'], ans_TM['D3']), color='red', linewidth=2, linestyle='dashed')
ax.grid(True)
print(file_name)

caique_plot(X_TM_disp, Y_TM_disp, plottype='scatter', annotate=False, result=result)